# Array programming in NumPy

While many of the array operations that are familiar to you from Python lists will also work with NumPy arrays, an important difference between NumPy arrays and Python lists is how we _should_ operate on them.  Specifically, to use NumPy arrays idiomatically, you sometimes need to think in terms of programming with arrays.  This means that some operations will have results that you don't expect if you're thinking of how Python lists work.

For example, consider the `max` function in Python:

In [ ]:
help(max)

So `max` returns its maximum argument.  If its arguments are lists, it will return the list with the largest first element (if there is a tie, it will return the list from the tie with the largest second element, and so on).  Try it out:

In [ ]:
max([1,1,5,9],[1,2,4,6])

The similarly-named NumPy `maximum` function has a slightly different interface and subtly-different behavior.  While the built-in `max` function will take the maximum from any number of arguments, NumPy's `maximum` takes two.  Furthermore, its result is slightly different, as you can see.

In [ ]:
import numpy as np
np.maximum([1,1,5,9],[1,2,4,6])

(Notice that we invoked `np.maximum` on two Python lists, but it would behave similarly on two NumPy arrays.)

In [ ]:
np.maximum(np.array([1,1,5,9]),np.array([1,2,4,6]))

Operations that work on entire arrays and can change an element at a time (rather than that work on entire arrays but only  as atomic objects) are extremely important in NumPy.

These elementwise operations are useful and they're also far more efficient than code you'd write in Python.  To see why, write a Python function to take the elementwise maximum of two arrays and compare its performance to `np.maximum`.   First, let's set up some test data:

In [ ]:
np.random.seed(0xDA7ABA5E)
ls1 = list(np.random.randint(256, size=5000))
ls2 = list(np.random.randint(256, size=5000))

In [ ]:
%%time

def nmax(la, lb):
    # FIXME: replace this function's body with a function that computes 
    # and returns the elementwise maximum of a and b.  Do not use NumPy.
    return max(la,lb)

for i in range(10000):
    nmax(ls1, ls2)

If you get stuck on the previous cell, [see the hint](hints/hint02.ipynb).

In [ ]:
%%time

def pmax(a, b):
    return np.maximum(a, b)

for i in range(10000):
    pmax(ls1, ls2)

We can see another example of array programming in the next two cells.  Assuming `ls` is a Python list and `na` is a NumPy array, how is `ls * 3`  different from `na * 3`?

In [ ]:
[1,2,5,9] * 3

In [ ]:
np.array([1,2,5,9]) * 3

What if we multiply a list by a list or an array by an array?  Try the next two cells to find out; we'll switch things up and do NumPy first.

In [ ]:
# Array-array multiplication
np.array([1,2,5,9]) * np.array([1,2,5,9])

In [ ]:
[1,2,5,9] * [1,2,5,9]

Write some Python code in the next cell that returns the same result as the NumPy code in 
the cell beginning with `# Array-array multiplication`.

(If you need a hint, the [hint for elementwise maximum](hints/hint02.ipynb) may be a good place to start!)

In [ ]:
def ewprod(l1, l2):
    # replace this function with one that computes 
    # the elementwise product of l1 and l2
    
    return l1

ewprod([1,2,5,9], [1,2,5,9])

We'll get a quick timing to see the performance difference between these approaches.  First, though, we'll make NumPy arrays from the random Python lists we used to test elementwise maximum.

In [ ]:
na1 = np.array(ls1)
na2 = np.array(ls2)

In [ ]:
%%time

acc = None

for i in range(10000):
    acc = na1 * na2

In [ ]:
%%time

acc = None

for i in range(10000):
    acc = ewprod(ls1, ls2)

As Python programmers, our instinct is to perform elementwise transformations on arrays with a `for` loop (in place) or with a list comprehension (making a copy).  The array programming features in NumPy enable us to perform these kinds of transformations _concisely_ (with less code) and _efficiently_ (with less time).  They are faster for several reasons: 

1. many of these library routines are implemented in native code (a `for` loop in C is often faster than a `for` loop in the Python interpreter),
2. some of these routines can use optimized primitives or hardware acceleration (depending on how you've installed NumPy), and
3. many of these routines can modify an argument in-place -- which concise pure-Python solutions cannot -- thus saving the cost of allocating a new array to store the result (and reducing memory pressure, memory fragmentation, and the workload for the garbage collector).

We'll see how to take advantage of modifying arrays in place in  the next cell.

In [ ]:
a = np.array([1,2,3])
b = np.array([3,4,5])
c = np.array([7,8,9])
np.maximum(a, b, c)

Remember that we said `maximum` takes the maximum of up to two arrays.  But in the previous cell we've given it three arguments and gotten a confusing result -- it seems to have ignored `c` and just taken the maximum of `a` and `b`.  What's going on?  Let's inspect `a`, `b`, and `c` and see if we can figure it out.

In [ ]:
print(a)
print(b)
print(c)

In the three-argument version of `maximum`, NumPy places the output in a preexisting array.  We can see that the contents of `c` are changed but its identity is not:

In [ ]:
a = np.array([1,2,3])
b = np.array([3,4,5])
c = np.array([7,8,9])

before = id(c)

np.maximum(a, b, c)

after = id(c)

print(c)
print(before == after)

It's okay to reuse one of the input arguments as the output argument, which is helpful if you're keeping a running aggregate:

In [ ]:
a = np.array([1,2,3,7])
b = np.array([3,1,5,2])

before = id(a)

np.maximum(a, b, a)

after = id(a)

print(a)
print(before == after)

You might be wondering how we can use elementwise multiplication to store a result in place, since there's no place for a third argument in the `*` operator. We'll need to use the `multiply` function in this case:

In [ ]:
a = np.array([1,1,2,3,5])
b = np.array([2,3,5,7,11])

np.multiply(a, b, a)
a

## Selecting and filtering arrays

In conventional Python, you might filter an array by adding a condition to a list comprehension.  So if you had some Fibonacci numbers, you could find the even ones like this:

In [ ]:
fibs = [1,1,2,3,5,8,13,21,34,55,89]
[f for f in fibs if f % 2 == 0]

In NumPy, we have a different way to filter arrays by applying a predicate to every element of an array and generating a *boolean mask*, or an array of truth values indicating whether or not the predicate is satisfied.

In [ ]:
a = np.array([1,1,2,3,5,8,13,21,34,55,89])
a % 2 == 0

NumPy provides functions that use these masks to filter and transform entire arrays at once.  For example, the `extract` function takes a mask (which coincidentally looks like a conditional expression) and returns an array consisting of only the elements that satisfy the expression.

In [ ]:
np.extract(a % 2 == 0, a)

We can also count the number of elements satisfying a conditional expression by summing the `True` values in the boolean mask:

In [ ]:
np.sum(a % 2 == 0)

Notice that Python's built-in `sum` function works in this example, but don't use it for this application in general — it has some subtle differences to `numpy.sum`.

In [ ]:
sum(a % 2 == 0)

There are several other functions that work similarly, like `where`, which takes a boolean  mask and two arrays of the same length.  `where` returns an array consisting of the elements from either the second or third arguments:  for elements in the condition that are true, the result will have the corresponding element from the second argument; for other elements, the result will have the corresponding element from the third argument.

In [ ]:
np.where(a % 2 == 0, a, np.zeros(len(a), int))

It's a little clunky to have that `np.zeros(len(a), int)`, isn't it?  I bet we can do better.

In [ ]:
np.where(a % 2 == 0, a, 0)

Wait a minute — `where` is supposed to take three arrays, but here we're giving it two arrays and an `int`.  What's going on?  Can we always substitute numbers for arrays?

In [ ]:
np.array([2,4,6,8]) * 2

It looks like we can!  What about substituting a shorter array for an array of the appropriate length — does that work?

In [ ]:
np.array([2,4,6,8]) * np.array([2, 2])

This doesn't work — we get an error:

`ValueError: operands could not be broadcast together with shapes (4,) (2,)`

To understand what this means, we'll have to talk a bit more about the words "broadcast" and "shape."

## Broadcasting

The *shape* of a NumPy array is a representation of its dimensions.  We've only talked about one-dimensional arrays so far, so the shape of one of the arrays we've seen is simply a 1-tuple containing the number of elements it has.

In [ ]:
np.array([1,2,3,4]).shape

In [ ]:
np.array([1,2,3,4,5,6]).shape

*Broadcasting* is how NumPy deals with function calls that expect arguments of the same shape but receive arguments of varying shapes.  